In [ ]:
%pip install langchain openai 'pydantic==1.10.12'

## Set up model

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
openai_model_name = "gpt-3.5-turbo"

llm_kwargs = dict(
    model_name=openai_model_name,
    temperature = 0.3,
    model_kwargs=dict(
        # top_p=0.6, 
        frequency_penalty=0.1
    ),
)
chat_model = ChatOpenAI(**llm_kwargs)

## Simple example

Does a job posting support remote work?

In [ ]:
# Set up prompt

from langchain.prompts import ChatPromptTemplate

template_string = "What is the remote work policy for the following job posting: \n{job_posting}"
prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
# https://www.thoughtspot.com/job/5357569?gh_jid=5357569
job_posting = """
VP, Information Security

Remote, USA

ThoughtSpot is a pioneering company at the forefront of the data analytics and business intelligence industry. Our platform empowers organizations to transform their data into actionable insights, enabling them to make smarter, data-driven decisions. As we continue to grow and innovate, we are seeking a highly skilled and experienced VP of Information Security to lead our efforts in safeguarding our systems, data, and operations.

We are seeking an accomplished and strategic VP of Information Security to lead our comprehensive security program at ThoughtSpot. The successful candidate will be responsible for overseeing the development, implementation, and management of all security-related initiatives to protect our assets, customers, employees, and partners. This role requires a dynamic individual with strong leadership, technical expertise, and a deep understanding of cybersecurity best practices.

Responsibilities: 

    Develop and implement a forward-thinking security strategy aligned with ThoughtSpot's business objectives and industry best practices.
    Identify, assess, and prioritize security risks, and develop strategies to mitigate potential threats to the organization.
    Lead efforts to protect sensitive data, ensuring compliance with relevant regulations (e.g., GDPR, HIPAA) and industry standards (e.g., SOC2, ISO 27001).
    Oversee the design and implementation of security measures for our cloud-based infrastructure and on-premises solutions.
    Manage the incident response plan to address and mitigate security incidents promptly and efficiently.
    Establish and maintain a culture of security awareness and education among employees, promoting a strong security-first mindset.
    Manage vulnerability assessment and remediation processes to identify and address potential security weaknesses.
    Ensure ThoughtSpot's compliance with relevant security and privacy regulations and certifications.
    Define and enforce security policies, standards, and procedures across the organization.
    Collaborate with internal and external auditors to conduct security audits and assessments, ensuring compliance with security controls.
    Build and lead a high-performing security team, providing mentorship, guidance, and professional development opportunities.
    Evaluate and manage security risks associated with third-party vendors and partners.
    Stay abreast of the latest cybersecurity threats, trends, and technologies, adapting security strategies as needed.

Qualifications:

    Bachelor's or Master's degree in Cybersecurity, Information Technology, or a related field. Relevant certifications (CISSP, CISM, CISA) are a plus.
    Proven experience in a senior leadership role within cybersecurity, preferably in a technology-focused organization.
    In-depth knowledge of cybersecurity principles, practices, technologies, and regulations.
    Strong track record of designing and implementing effective security strategies and programs.
    Exceptional communication skills, with the ability to convey complex security concepts to both technical and non-technical stakeholders.
    Experience managing and leading diverse security teams.
    Strong problem-solving skills and the ability to make informed decisions under pressure.
    Collaborative and results-driven approach to leadership.
    Familiarity with cloud security, network security, application security, and endpoint security best practices.

Join Our Team:

If you are a visionary leader with a passion for cybersecurity and a drive to protect valuable assets, we invite you to join our team at ThoughtSpot. As the Head of Security, you will play a pivotal role in shaping our security posture and ensuring the continued success and growth of our organization."""

In [ ]:
response = chat_model(prompt_template.format_messages(job_posting=job_posting))

In [ ]:
response.content

## Parsing outputs using openai functions

In [ ]:
# Define general output extraction prompt

from langchain.prompts import HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage

prompt_msgs = [
    SystemMessage(
        content="You are a world class algorithm for extracting information in structured formats."
    ),
    HumanMessage(
        content="Use the given format to extract information from the following input:"
    ),
    HumanMessagePromptTemplate.from_template("{input}"),
    HumanMessage(content="Tips: Make sure to answer in the correct format."),
]
extractor_prompt = ChatPromptTemplate(messages=prompt_msgs)

In [ ]:
# Define output format using Pydantic

from pydantic import BaseModel, Field
from typing import List

# Pydantic class
class Job_Description(BaseModel):
    """Description of a job posting"""

    company: str = Field(
        ..., description="Name of the company that has the job opening"
    )
    job_title: str = Field(..., description="Job title")
    team: str = Field(
        ...,
        description="Name of the team within the company. Team name should be null if it's not known.",
    )
    job_summary: str = Field(
        ..., description="Brief summary of the job, not exceeding 100 words"
    )
    salary: str = Field(
        ...,
        description="Salary amount or range. Salary should be null if it's not known.",
    )
    duties: List[str] = Field(
        ...,
        description="The role, responsibilities and duties of the job as an itemized list, not exceeding 500 words",
    )
    qualifications: List[str] = Field(
        ...,
        description="The qualifications, skills, and experience required for the job as an itemized list, not exceeding 500 words",
    )
    is_fully_remote: bool = Field(
        ...,
        description="Does the job have an option to work fully (100%) remotely? Hybrid or partial remote is marked as `False`. Use `None` if the answer is not known.",
    )

In [ ]:
# Create chain using openai functions

from langchain.chains.openai_functions import create_structured_output_chain

extractor_chain = create_structured_output_chain(
    output_schema=Job_Description, llm=chat_model, prompt=extractor_prompt
)

In [ ]:
# Run the chain and get parsed output
import yaml

response = extractor_chain.predict(input=job_posting)
parsed_job = response.dict()
print(yaml.dump(parsed_job))

## Chain of thought

Get suggestions for improving Resume applying to the job posting

In [ ]:
# Define prompt
# Prompt includes
# - Steps for LLM to follow
# - Separator tokens, like <> and backticks for delineating steps and inputs

prompt_msgs = [
    SystemMessage(
        content=(
            "You are an expert Resume Reviewer proficient in making suggestions for improvements to a Resume. You strictly follow the provided steps."
        )
    ),
    HumanMessage(
        content=(
            "Your goal is to read and understand both a job posting and my Resume, identify the requirements from the job posting that are missing in my Resume, and finally make suggestions for improvements."
        )
    ),
    HumanMessage(
        content=(
            "\nWe will follow the following steps:"
            "\nStep 1: I will give you a job posting."
            "\nStep 2: Then I will give you my Resume."
            "\nStep 3: You must create an internal list of those requirements from the job posting from Step 1 that are NOT present in my Resume from Step 2. Follow these sub-steps:"
            "\n   Sub-Step 3a: Identify an internal list of those requirements from the job posting that are NOT present in my Resume."
            "\n   Sub-Step 3b: For each missing requirement that you identified in Sub-Step 3a, check again whether that requirement is in fact present in my Resume. Note that a requirement may be present in my Resume in a different phrasing than in the job posting. If the requirement is present in my Resume, remove it from your list of missing requirements."
            "\n   Sub-Step 3c: Sub-steps are complete. Move to the next Step."
            "\nStep 4: You must finally create a list of suggestions for improving my Resume based on the final list of missing requirements from the previous step, and the job posting."
        )
    ),
    HumanMessage(content=("Let us begin...")),
    HumanMessage(
        content="Step 1: I am providing the job posting, which includes two sections about the job - <Duties>, <Qualifications>. The entire job posting is enclosed in three backticks:"
    ),
    HumanMessagePromptTemplate.from_template(
        "```\nJob Posting:"
        "\n<Duties>\n{duties}\n"
        "\n<Qualifications>\n{qualifications}\n"
        "```"
    ),
    HumanMessage(
        content="Step 2: I am providing my Resume enclosed in four backticks:"
    ),
    HumanMessagePromptTemplate.from_template(
        "````\nMy Resume:"
        "\n{resume}\n"
        "````"
    ),
    HumanMessage(
        content="You must now complete the rest of the steps starting at Step 3, including any sub-steps."
    ),
    HumanMessage(
        content="Tips: Make sure to answer in the correct format, and stick to any word or item limits."
    ),
]
improver_prompt = ChatPromptTemplate(messages=prompt_msgs)

In [ ]:
# Define output format and chain
class Resume_Improvements(BaseModel):
    improvements: List[str] = Field(
        ..., description="List of suggestions for improvement"
    )

improver_chain = create_structured_output_chain(
    output_schema=Resume_Improvements, llm=chat_model, prompt=improver_prompt
)

In [ ]:
# input resume
resume = """
    Nicole Perlroth spent a decade as the lead cybersecurity reporter at The New York Times. Her investigations rooted out Russian hacks of nuclear plants, airports, elections, and petrochemical plants; North Korea's cyberattack against Sony Pictures, Bangladesh banks and crypto exchanges; Iranian attacks on oil companies, banks and dams; and thousands of Chinese cyberattacks against American businesses, including leading the investigation of the months-long Chinese hack of The Times. Her outing of hacking divisions within China’s PLA compelled the first United States hacking charges against the Chinese military and earned her the prestigious “Best in Business Award” from the Society of American Business Editors and Writers. She left the Times in 2021 to join the Department of Homeland Security’s Cybersecurity Advisory Committee.
    She is the author of the New York Times bestselling book “This Is How They Tell Me The World Ends,” about the global cyber arms race, which won the McKinsey and Financial Times’ 2021 “Business Book of the Year Award.” The book was optioned by Tommy Schlamme for FX Networks and several of her Times articles have been optioned for television. 
    Ms. Perlroth has been widely cited and published, beyond The Times in The New Yorker, The New York Review of Books, The Economist, Wired, Forbes, CNN, PBS, NPR, Bloomberg, The Wall Street Journal, The Washington Post, NBC’s “Meet The Press,” “The Rachel Maddow Show,” “Dan Rather’s America,” Axios, CBS, CNBC, and Lawfare, as well as The Times’ “The Daily” and “Sway” podcasts, VOX’s “Pivot” podcast, among others.
    She has delivered keynotes and speeches for TED, the State Department, the World Bank, the Munich Security Conference, RSA, the Council on Foreign Relations, World Affairs Council, Metropolitan Club, the Stockholm Forum on Peace and Development, How To Academy, In-Q-Tel, Track ii Diplomacy, the Friedrich Naumann Foundation Defense Policy Advisors, Hack the Capitol, the Center for European Policy Analysis and the CIOSExchange, an invite-only gathering of Fortune 50 Chief Information Officers.
    She also lectures at Stanford, including the Stanford Graduate School of Business, Princeton University, Columbia, Cornell, Georgetown, Harvard Kennedy School, Hult International Business School, The Fletcher School, Naval War College, Fordham Law, Berkeley, John Hopkins and was selected as the inaugural “Journalist in Residence” for the University of Texas Strauss World Affairs program. 
    Ms. Perlroth is a graduate of Princeton University (B.A.), Stanford University (M.A.).
    """

In [ ]:
# Run the chain and print outputs

# format prompt inputs so list items are separated by linebreaks
prompt_imputs = dict(
    duties = '\n    - '.join(parsed_job['duties']),
    qualifications = '\n    - '.join(parsed_job['qualifications']),
    resume = resume
)
response = improver_chain.predict(**prompt_imputs)
print(yaml.dump(response.dict()))

## Modified zero-shot chain of thought

1. Provide instructions and a broad outline of steps.
2. Ask the LLM to determine a full list of steps and follow them.

In [ ]:
# Define the prompt
prompt_msgs = [
    SystemMessage(
        content=(
            "You are an expert critic. "
            "Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>."
        )
    ),
    HumanMessagePromptTemplate.from_template(
        "<Job Posting>"
        "\nThe ideal candidate is able to perform the following duties:{duties}\n"
        "\nThe ideal candidate has the following qualifications:{qualifications}\n"
    ),
    HumanMessagePromptTemplate.from_template(
        "<Resume>"
        "\n{resume}\n"
    ),
    HumanMessage(
        content="<Instruction> Critique my <Resume>, and create a list of suggestions how I can improve it "
        "so it showcases that I am the ideal candidate who meets all the requirements of the <Job Posting>."
    ),
    HumanMessage(
        content=(
            "<Steps>"
            "\n- Create a <Plan> for following the <Instruction> while meeting all the <Criteria>."
            "\n- What <Additional Steps> are needed to follow the <Plan>?"
            "\n- Follow all steps one by one and show your <Work>."
            "\n- Verify that all improvements have been identified, and there are no spelling errors. Update the <Work> if necessary."
            "\n- Provide the answer to the <Instruction> with prefix <Final Answer>."
        )
    ),
]
better_improver_prompt = ChatPromptTemplate(messages=prompt_msgs)

In [ ]:
# Run the chain without output parsing due to complexity of the output
from langchain import LLMChain
better_improver_chain = LLMChain(
    llm=chat_model,
    prompt=better_improver_prompt,
    verbose=True
)
better_improver_chain_response = better_improver_chain.predict(**prompt_imputs)

In [ ]:
print(better_improver_chain_response)

## Parse CoT output

In [ ]:
# Define output format and run chain
class Resume_Improver_Output(BaseModel):
    plan: List[str] = Field(..., description="itemized <Plan>")
    additional_steps: List[str] = Field(..., description="itemized <Additional Steps>")
    work: List[str] = Field(..., description="itemized <Work>")
    final_answer: List[str] = Field(
        ..., description="<Final Answer> in the correct format"
    )
extractor_chain = create_structured_output_chain(
    output_schema=Resume_Improver_Output, llm=chat_model, prompt=extractor_prompt
)
improvements = extractor_chain.predict(input=better_improver_chain_response)

In [ ]:
print(yaml.dump(improvements.dict()))